## Final Project Submission

Please fill out:
* Student name: 
* Student pace: self paced / part time / full time
* Scheduled project review date/time: 
* Instructor name: 
* Blog post URL:


In [1]:
import numpy as np
import pandas as pd
from scipy import stats
import datetime
datetime.datetime.strptime

import matplotlib.pyplot as plt
from matplotlib import ticker
%matplotlib inline
import seaborn as sns
plt.style.use('seaborn-notebook')# Your code here - remember to use markdown cells for comments as well!

In [2]:
kc_house_df = pd.read_csv('./data/kc_house_data.csv', parse_dates=['date']) # specify index_col=0 to avoid creating an "Unnamed: 0" column.

# specify parse date to avoid object dtype for dates

kc_house_df

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,2014-10-13,221900.0,3,1.00,1180,5650,1.0,NaN,NONE,...,7 Average,1180,0.0,1955,0.0,98178,47.5112,-122.257,1340,5650
1,6414100192,2014-12-09,538000.0,3,2.25,2570,7242,2.0,NO,NONE,...,7 Average,2170,400.0,1951,1991.0,98125,47.7210,-122.319,1690,7639
2,5631500400,2015-02-25,180000.0,2,1.00,770,10000,1.0,NO,NONE,...,6 Low Average,770,0.0,1933,NaN,98028,47.7379,-122.233,2720,8062
3,2487200875,2014-12-09,604000.0,4,3.00,1960,5000,1.0,NO,NONE,...,7 Average,1050,910.0,1965,0.0,98136,47.5208,-122.393,1360,5000
4,1954400510,2015-02-18,510000.0,3,2.00,1680,8080,1.0,NO,NONE,...,8 Good,1680,0.0,1987,0.0,98074,47.6168,-122.045,1800,7503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21592,263000018,2014-05-21,360000.0,3,2.50,1530,1131,3.0,NO,NONE,...,8 Good,1530,0.0,2009,0.0,98103,47.6993,-122.346,1530,1509
21593,6600060120,2015-02-23,400000.0,4,2.50,2310,5813,2.0,NO,NONE,...,8 Good,2310,0.0,2014,0.0,98146,47.5107,-122.362,1830,7200
21594,1523300141,2014-06-23,402101.0,2,0.75,1020,1350,2.0,NO,NONE,...,7 Average,1020,0.0,2009,0.0,98144,47.5944,-122.299,1020,2007
21595,291310100,2015-01-16,400000.0,3,2.50,1600,2388,2.0,NaN,NONE,...,8 Good,1600,0.0,2004,0.0,98027,47.5345,-122.069,1410,1287


In [3]:
kc_house_df.drop_duplicates(subset='id', inplace=True)

In [4]:
kc_house_df.corr()['price']

id              -0.018525
price            1.000000
bedrooms         0.309453
bathrooms        0.526229
sqft_living      0.701875
sqft_lot         0.089111
floors           0.256286
sqft_above       0.605294
yr_built         0.052906
yr_renovated     0.130341
zipcode         -0.052371
lat              0.305744
long             0.020983
sqft_living15    0.584549
sqft_lot15       0.082438
Name: price, dtype: float64

In [5]:
kc_house_df['id'].value_counts()

2911700010    1
5450300010    1
5104511600    1
1160000115    1
686530110     1
             ..
2115510470    1
2922701305    1
6071600370    1
526059224     1
1777500160    1
Name: id, Length: 21420, dtype: int64

In [6]:
kc_house_df.isnull().sum()

id                  0
date                0
price               0
bedrooms            0
bathrooms           0
sqft_living         0
sqft_lot            0
floors              0
waterfront       2353
view               63
condition           0
grade               0
sqft_above          0
sqft_basement       0
yr_built            0
yr_renovated     3804
zipcode             0
lat                 0
long                0
sqft_living15       0
sqft_lot15          0
dtype: int64

In [7]:
kc_house_df['waterfront'].fillna('no', inplace=True)
kc_house_df['yr_renovated'].fillna(0, inplace=True)
kc_house_df['view'].fillna('NONE', inplace=True)

In [8]:
kc_house_df.isnull().sum()

id               0
date             0
price            0
bedrooms         0
bathrooms        0
sqft_living      0
sqft_lot         0
floors           0
waterfront       0
view             0
condition        0
grade            0
sqft_above       0
sqft_basement    0
yr_built         0
yr_renovated     0
zipcode          0
lat              0
long             0
sqft_living15    0
sqft_lot15       0
dtype: int64

In [9]:
kc_house_df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,2014-10-13,221900.0,3,1.00,1180,5650,1.0,no,NONE,...,7 Average,1180,0.0,1955,0.0,98178,47.5112,-122.257,1340,5650
1,6414100192,2014-12-09,538000.0,3,2.25,2570,7242,2.0,NO,NONE,...,7 Average,2170,400.0,1951,1991.0,98125,47.7210,-122.319,1690,7639
2,5631500400,2015-02-25,180000.0,2,1.00,770,10000,1.0,NO,NONE,...,6 Low Average,770,0.0,1933,0.0,98028,47.7379,-122.233,2720,8062
3,2487200875,2014-12-09,604000.0,4,3.00,1960,5000,1.0,NO,NONE,...,7 Average,1050,910.0,1965,0.0,98136,47.5208,-122.393,1360,5000
4,1954400510,2015-02-18,510000.0,3,2.00,1680,8080,1.0,NO,NONE,...,8 Good,1680,0.0,1987,0.0,98074,47.6168,-122.045,1800,7503


In [10]:
kc_house_df.drop(columns= ['lat', 'long'], inplace=True)

In [11]:
#creating reno column
kc_house_df['renovated'] = 0
kc_house_df.loc[kc_house_df['yr_renovated'] > 1, 'renovated'] = 1

#creating recent reno column
kc_house_df['recent_reno'] = 0
kc_house_df.loc[kc_house_df['yr_renovated'] > 2000, 'recent_reno'] = 1

In [12]:
kc_house_df

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,sqft_living15,sqft_lot15,renovated,recent_reno
0,7129300520,2014-10-13,221900.0,3,1.00,1180,5650,1.0,no,NONE,...,7 Average,1180,0.0,1955,0.0,98178,1340,5650,0,0
1,6414100192,2014-12-09,538000.0,3,2.25,2570,7242,2.0,NO,NONE,...,7 Average,2170,400.0,1951,1991.0,98125,1690,7639,1,0
2,5631500400,2015-02-25,180000.0,2,1.00,770,10000,1.0,NO,NONE,...,6 Low Average,770,0.0,1933,0.0,98028,2720,8062,0,0
3,2487200875,2014-12-09,604000.0,4,3.00,1960,5000,1.0,NO,NONE,...,7 Average,1050,910.0,1965,0.0,98136,1360,5000,0,0
4,1954400510,2015-02-18,510000.0,3,2.00,1680,8080,1.0,NO,NONE,...,8 Good,1680,0.0,1987,0.0,98074,1800,7503,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21592,263000018,2014-05-21,360000.0,3,2.50,1530,1131,3.0,NO,NONE,...,8 Good,1530,0.0,2009,0.0,98103,1530,1509,0,0
21593,6600060120,2015-02-23,400000.0,4,2.50,2310,5813,2.0,NO,NONE,...,8 Good,2310,0.0,2014,0.0,98146,1830,7200,0,0
21594,1523300141,2014-06-23,402101.0,2,0.75,1020,1350,2.0,NO,NONE,...,7 Average,1020,0.0,2009,0.0,98144,1020,2007,0,0
21595,291310100,2015-01-16,400000.0,3,2.50,1600,2388,2.0,no,NONE,...,8 Good,1600,0.0,2004,0.0,98027,1410,1287,0,0


In [13]:
sqft_filter = kc_house_df[(kc_house_df['sqft_living'] > 1800) & (kc_house_df['sqft_living'] < 2800)]
sqft_filter

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,sqft_living15,sqft_lot15,renovated,recent_reno
1,6414100192,2014-12-09,538000.0,3,2.25,2570,7242,2.0,NO,NONE,...,7 Average,2170,400.0,1951,1991.0,98125,1690,7639,1,0
3,2487200875,2014-12-09,604000.0,4,3.00,1960,5000,1.0,NO,NONE,...,7 Average,1050,910.0,1965,0.0,98136,1360,5000,0,0
9,3793500160,2015-03-12,323000.0,3,2.50,1890,6560,2.0,NO,NONE,...,7 Average,1890,0.0,2003,0.0,98038,2390,7570,0,0
14,1175000570,2015-03-12,530000.0,5,2.00,1810,4850,1.5,NO,NONE,...,7 Average,1810,0.0,1900,0.0,98107,1360,4850,0,0
16,1875500060,2014-07-31,395000.0,3,2.00,1890,14040,2.0,NO,NONE,...,7 Average,1890,0.0,1994,0.0,98019,1890,14018,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21575,4140940150,2014-10-02,572000.0,4,2.75,2770,3852,2.0,NO,NONE,...,8 Good,2770,0.0,2014,0.0,98178,1810,5641,0,0
21578,5087900040,2014-10-17,350000.0,4,2.75,2500,5995,2.0,no,NONE,...,8 Good,2500,0.0,2008,0.0,98042,2530,5988,0,0
21587,7852140040,2014-08-25,507250.0,3,2.50,2270,5536,2.0,no,NONE,...,8 Good,2270,0.0,2003,0.0,98065,2270,5731,0,0
21589,3448900210,2014-10-14,610685.0,4,2.50,2520,6023,2.0,NO,NONE,...,9 Better,2520,0.0,2014,0.0,98056,2520,6023,0,0


In [14]:
date_filter = kc_house_df[((kc_house_df['date'] > "2001-01-01" ) & (kc_house_df['date'] < "2015-12-31"))]
date_filter

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,sqft_living15,sqft_lot15,renovated,recent_reno
0,7129300520,2014-10-13,221900.0,3,1.00,1180,5650,1.0,no,NONE,...,7 Average,1180,0.0,1955,0.0,98178,1340,5650,0,0
1,6414100192,2014-12-09,538000.0,3,2.25,2570,7242,2.0,NO,NONE,...,7 Average,2170,400.0,1951,1991.0,98125,1690,7639,1,0
2,5631500400,2015-02-25,180000.0,2,1.00,770,10000,1.0,NO,NONE,...,6 Low Average,770,0.0,1933,0.0,98028,2720,8062,0,0
3,2487200875,2014-12-09,604000.0,4,3.00,1960,5000,1.0,NO,NONE,...,7 Average,1050,910.0,1965,0.0,98136,1360,5000,0,0
4,1954400510,2015-02-18,510000.0,3,2.00,1680,8080,1.0,NO,NONE,...,8 Good,1680,0.0,1987,0.0,98074,1800,7503,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21592,263000018,2014-05-21,360000.0,3,2.50,1530,1131,3.0,NO,NONE,...,8 Good,1530,0.0,2009,0.0,98103,1530,1509,0,0
21593,6600060120,2015-02-23,400000.0,4,2.50,2310,5813,2.0,NO,NONE,...,8 Good,2310,0.0,2014,0.0,98146,1830,7200,0,0
21594,1523300141,2014-06-23,402101.0,2,0.75,1020,1350,2.0,NO,NONE,...,7 Average,1020,0.0,2009,0.0,98144,1020,2007,0,0
21595,291310100,2015-01-16,400000.0,3,2.50,1600,2388,2.0,no,NONE,...,8 Good,1600,0.0,2004,0.0,98027,1410,1287,0,0
